In [1]:
import gym
import gym_tictactoe
import numpy as np

In [2]:
def checkRows(board):
    for row in board:
        if len(set(row)) == 1:
            return row[0]
    return 0

def checkDiagonals(board):
    if len(set([board[i][i] for i in range(len(board))])) == 1:
        return board[0][0]
    if len(set([board[i][len(board)-i-1] for i in range(len(board))])) == 1:
        return board[0][len(board)-1]
    return 0

def checkWin(board):
    #transposition to check rows, then columns
    for newBoard in [board, np.transpose(board)]:
        result = checkRows(newBoard)
        if result:
            return result
    return checkDiagonals(board)

## Basic adversary for the RL algorithm to compete against

In [3]:
class ImperfectAgent():
    def policy(self, state):
        best_action = None
        
        if state[4] is 0:
            best_action = 4
        elif state[0] is 0:
            best_action = 0
        elif state[2] is 0:
            best_action = 2
        elif state[6] is 0:
            best_action = 6
        elif state[8] is 0:
            best_action = 8
        elif state[1] is 0:
            best_action = 1
        elif state[3] is 0:
            best_action = 3
        elif state[5] is 0:
            best_action = 5
        elif state[7] is 0:
            best_action = 7
        else:
            print('invalid state')
            raise "invalid state"
            
        return np.eye(9)[best_action]

In [4]:
env = gym.make('TicTacToe-v1')
env.init(symbols=[1, 2]) # Define users symbols

/home/nef/anaconda3/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Environment '<class 'gym_tictactoe.tic_tac_toe.TicTacToeEnv'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [5]:
adversary = ImperfectAgent()

In [6]:
original_state = env.reset()

## RL algorithm based agent using temporal-difference (TD) learning

In [7]:
from sympy.utilities.iterables import multiset_permutations
import random

In [95]:
class TDLearningAgent():
    def __init__(self, epsilon=0.1, epsilon_decay=0.01, playerNum=2, oppNum=1, lr=0.2, action_space=9):
        self.valueFunctionTable = np.full((19683,), 0.5)
        self.stateHistories = []
        self.player = playerNum
        self.opponent = oppNum
        self.learningRate = lr
        self.num_squares = action_space
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        
        for index, _ in enumerate(self.valueFunctionTable):
            winner = checkWin( np.array(self.num_to_state(index)).reshape( (3,3) ) )
            
            if winner == self.player:
                self.valueFunctionTable[index] = 1
            elif winner == self.opponent:
                self.valueFunctionTable[index] = 0
        
    
    def state_to_num(self, state):
        assert len(state) == 9
        return np.ravel_multi_index(state, (3, 3, 3, 3, 3, 3, 3, 3, 3))
    
    def num_to_state(self, num):
        return np.unravel_index(num, (3, 3, 3, 3, 3, 3, 3, 3, 3))
        
    def value_from_state(self, state):
        curr_index = self.state_to_num(state)
        return self.valueFunctionTable[curr_index]
        
    def train(self):     
        
        
        for i in range(len(self.stateHistories)-1, 1, -1):
            curr_state = self.stateHistories[i-1]
            next_state = self.stateHistories[i]
            
            newStateValue = self.valueFunctionTable[curr_state] + self.learningRate*(self.valueFunctionTable[next_state] - self.valueFunctionTable[curr_state])
            
            self.valueFunctionTable[curr_state] = newStateValue
        self.stateHistories = []
        
        # decay epsilon
        self.epsilon = self.epsilon * (1 - self.epsilon_decay)
            
    def epsilon_greedy_policy(self, state):
        if random.random() < self.epsilon:
            found = False
            while not found:
                action = int(random.randint(0, self.num_squares - 1))
                if state[action] == 0:
                    found = True
                    
            return np.eye(9)[action]
        else: return self.policy(state)
        
    def policy(self, state):        
        value_predictions = []
        
        for i, element in enumerate(state):
            foward_state = state.copy()
            if element is 0:
                foward_state[i] = self.player
                value_prediction = self.value_from_state(foward_state)
                value_predictions.append(value_prediction)
            else:
                value_predictions.append(-1)
                
        greedy_action = np.argmax(value_predictions) 

        
        return np.eye(9)[greedy_action]

In [97]:
Imperfect = ImperfectAgent()
TDLearner = TDLearningAgent(playerNum=2, oppNum=1)
TDLearnerAdversary = TDLearningAgent(playerNum=1, oppNum=2)

show_every = 10_000

for gameNum in range(1, 100_000):
    
    if gameNum % show_every == 0:
        print(gameNum)
    
    
    done = False
    state = env.reset()
    curr_action = 1
    TDLearner.stateHistories.append(TDLearner.state_to_num(state))
    if  gameNum % show_every == 0:
            env.render(mode=None)
    while not done:
        
        
        
        if curr_action == 1:
            decision = TDLearner.epsilon_greedy_policy(state)
        else:
            decision = TDLearnerAdversary.epsilon_greedy_policy(state)
        
        actionIndex = np.random.choice(9, 1, p=decision)[0]
                
        _state, reward, done, info = env.step(actionIndex, curr_action)
        
        if (done):            
            winner = checkWin(np.array(_state).reshape((3,3)))
        
        if gameNum % show_every == 0:
            env.render(mode=None)
    
        state = _state
        
        if curr_action == 1:
            curr_action = 2
        else:
            curr_action = 1
        TDLearner.stateHistories.append(TDLearner.state_to_num(state))
    TDLearner.train()

10000
 -------------
 |   |   |   |
 -------------
 |   |   |   |
 -------------
 |   |   |   |
 -------------

 -------------
 | x |   |   |
 -------------
 |   |   |   |
 -------------
 |   |   |   |
 -------------

 -------------
 | x | o |   |
 -------------
 |   |   |   |
 -------------
 |   |   |   |
 -------------

 -------------
 | x | o | x |
 -------------
 |   |   |   |
 -------------
 |   |   |   |
 -------------

 -------------
 | x | o | x |
 -------------
 |   |   |   |
 -------------
 |   |   | o |
 -------------

 -------------
 | x | o | x |
 -------------
 | x |   |   |
 -------------
 |   |   | o |
 -------------

 -------------
 | x | o | x |
 -------------
 | x |   |   |
 -------------
 | o |   | o |
 -------------

 -------------
 | x | o | x |
 -------------
 | x |   |   |
 -------------
 | o | x | o |
 -------------

 -------------
 | x | o | x |
 -------------
 | x | o |   |
 -------------
 | o | x | o |
 -------------

 -------------
 | x | o | x |
 ---------

In [98]:
TDLearner.num_to_state(18315)

(2, 2, 1, 0, 1, 0, 1, 0, 0)

In [ ]:
TDLearner.value_from_state([2, 2, 1, 0, 1, 0, 1, 0, 0])